In [9]:
import os, collections
import numpy as np
import pandas as pd

#from gtfparse import read_gtf

ONTO_PATH = "/data2/zhoujb/project/cowpea_project/rawData/geneOntology/"

In [32]:
chr_map = {1:"Chr01", 2:"Chr02", 3:"Chr03", 
           4:"Chr04", 5:"Chr05", 6:"Chr06", 
           7:"Chr07", 8:"Chr08", 9:"Chr09", 
           '9':"Chr09", '10':"Chr10", '11':"Chr11"}

raw_table = pd.read_table(os.path.join(ONTO_PATH, "GCF_004118075.2_ASM411807v2_feature_table.txt"))
raw_table_gene = raw_table[(raw_table['# feature']=="gene")&(raw_table["seq_type"]=="chromosome")].copy()
raw_table_gene["Chr"] = raw_table_gene["chromosome"].map(chr_map)

raw_table_gene[['Chr', 'start', 'end', 'symbol', 'GeneID']].to_csv(os.path.join(ONTO_PATH, "feature_table_gene_id.txt"), sep="\t", index=0)

/tmp/ipykernel_364638/476523985.py:6: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_table = pd.read_table(os.path.join(ONTO_PATH, "GCF_004118075.2_ASM411807v2_feature_table.txt"))


In [29]:
go_id2name_d = {}
with open(os.path.join(ONTO_PATH, "go-basic.obo")) as f:
    for line in f:
        if line.startswith("[Term]"):
            go_id = f.readline().strip()[4:]
            go_name = f.readline().strip()[6:]
            go_id2name_d[go_id] = go_name

raw_gaf = pd.read_table(os.path.join(ONTO_PATH, "GCF_004118075.2_ASM411807v2_gene_ontology.gaf"), skiprows=8)

with open(os.path.join(ONTO_PATH, "go_gene_id.txt"), "w") as out_f:
    print("\t".join(["GO_TYPE", "GO_ID", "GO_NAME", "Term"]), file=out_f)

    for group_aspect in raw_gaf.groupby("Aspect"):
        group_aspect_name, group_aspect_data = group_aspect
        for group_go in group_aspect_data.groupby("GO_ID"):
            group_go_name, group_go_data = group_go
            print("\t".join([group_aspect_name, group_go_name, go_id2name_d[group_go_name], 
                             ",".join(group_go_data["Symbol"].unique())]), file=out_f)
print("ALL DONE")

ALL DONE


In [46]:
kegg_id2name_d = {}
with open(os.path.join(ONTO_PATH, "vun_pathway_name_kegg")) as f:
    for line in f:
        line = line.rstrip().split("\t")
        kegg_name = line[1].split(" - ")[0]
        kegg_id2name_d[line[0]] = kegg_name


kegg_id2list_d = collections.defaultdict(list)
with open(os.path.join(ONTO_PATH, "vun_pathway_list_kegg")) as f:
    for line in f:
        line = line.rstrip().split()
        kegg_id = line[0][5:]
        kegg_gene = line[1][4:]
        kegg_id2list_d[kegg_id].append(kegg_gene)

with open(os.path.join(ONTO_PATH, "kegg_gene_id.txt"), "w") as out_f:
    print("\t".join(["KEGG_ID", "KEGG_NAME", "Term"]), file=out_f)
    for key, value in kegg_id2list_d.items():
        print("\t".join([key, kegg_id2name_d[key], ",".join(value)]), file=out_f)

print("ALL DONE")

In [ ]:
# OLD
# returns GTF with essential columns such as "feature", "seqname", "start", "end"
# alongside the names of any optional keys which appeared in the attribute column
raw_gtf = read_gtf(os.path.join(ONTO_PATH, "GCF_004118075.2_ASM411807v2_genomic.gtf"), result_type="pandas")
gtf_gene = raw_gtf[raw_gtf["feature"] == "gene"].copy()

gtf_gene["GeneID"] = gtf_gene["db_xref"].str[7:]

gtf_gene[['start', 'end', 'gene_id', 'db_xref', 'GeneID']].to_csv(os.path.join(ONTO_PATH, "gtf_gene_id.txt"), sep="\t", index=0)